# Community Detection

In [Chapter 6](#link?) you learned how to estimate the parameters for an SBM via Maximum Likelihood Estimation (MLE). Unfortunately, we have skipped a relatively fundamental problem with SBM parameter estimation. You will notice that everything we have covered about the SBM to date assumes that you the assignments to one of $K$ possible communities for each node, which is given by the node assignment variable $z_i$ for each node in the network. Why is this problematic? Well, quite simply, when you are learning about *many* different networks you might come across, you might not actually *observe* the communities of each node.

Consider, for instance, the school example you have worked extensively with. In the context of the SBM, it makes sense to guess that two individuals will have a higher chaance of being friends if they attend the same school than if they did not go to the same school. Remember, when you knew what school each student was from and could *order* the students by school ahead of time, the network looked like this:

In [ ]:
from graspologic.simulations import sbm
import numpy as np

ns = [50, 50]  # network with 100 nodes
B = [[0.5, 0.2], [0.2, 0.5]]  # block matrix

# sample a single simple adjacency matrix from SBM(z, B)
A = sbm(n=ns, p=B, directed=False, loops=False)
zs = [1 for i in range(0, 50)] + [2 for i in range(0, 50)]

In [ ]:
from graphbook_code import draw_multiplot, cmaps
import matplotlib.pyplot as plt

draw_multiplot(A, labels=zs, title="$SBM_n(z, B)$ Simulation, nodes ordered by school");

The block structure is *completely obvious*, and it seems like you could almost just guess which nodes are from which communities by looking at the adjacency matrix (with the proper ordering). Ant therein lies the issue: if you did not know which school each student was from, you would have *no way* of actually using the technique you described in the preceding chapter to estimate parameters for your SBM. If your nodes were just randomly ordered, we might see something instead like this:

In [ ]:
# generate a reordering of the n nodes
vtx_perm = np.random.choice(np.sum(ns), size=np.sum(ns), replace=False)

Aperm = A[tuple([vtx_perm])] [:,vtx_perm]

In [ ]:
draw_multiplot(Aperm, title="$SBM_n(z, B)$ Simulation, random node order");

It is extremely unclear which nodes are in which community, because there is no immediate block structure to the network. So, what *can* you do?

What you will do is use a clever technique we discussed in [Chapter 6](#link?), called an *embedding*, to learn not only about each edge, but to *learn about each node in relation to all of the other nodes*. What do we mean by this? What we mean is that, while looking at a single edge in your network, you only have two possible choices: the edge exists or it does not exist. However, if you instead consider nodes in *relation* to one another, you can start to deduce patterns about how your nodes might be organized in the community sense. While seeing that two students Bill and Stephanie were friends will not tell us whether Bill and Stephanie were in the same school, if you knew that Bill and Stephanie also shared many other friends (such as Denise, Albert, and Kwan), and those friends also tended to be friends, that piece of information might tell us that they all might happen to be school mates. 

The embedding technique you will tend to employ, the *spectral embedding* from [Chapter 6](#link?), allows you to pick up on these *community* sorts of tendencies. When we call a set of nodes a **community** in the context of a network, what we mean is that these nodes tend to be more connected (more edges exist between and amongst them) than with other nodes in the network. The spectral embeddings will help us to identify these communities of nodes, and hopefully, when you review the communities of nodes you learn, you will be able to derive some sort of insight as to what, exactly, these communities are. For instance, in your school example, ideally, you might pick up on two communities of nodes, one for each school. The process of learning community assignments for nodes in a network is known as **community detection**.

## Stochastic Block Model with unknown communities when you know the number of communities $K$

When you know the number of communities (even if you don't know which community each node is in), the procedure for fitting a Stochastic Block Model to a network is relatively straightforward. Let's consider a similar example to the scenario you had [in the introduction](#link?), but with $3$ communities instead of $2$. You will have a block matrix given by:
\begin{align*}
    B &= \begin{bmatrix}
        0.4 & 0.2 & 0.2 \\
        0.2 & 0.4 & 0.2 \\
        0.2 & 0.2 & 0.4
    \end{bmatrix}
\end{align*}
Which is a Stochastic block model in which the within-community edge probability is $0.4$, and exceeds the between-community edge probability of $0.2$. You will produce a matrix with $120$ nodes in total.

In [ ]:
from graspologic.simulations import sbm

ns = [50, 40, 30]
B = [[0.6, 0.2, 0.2],
     [0.2, 0.6, 0.2],
     [0.2, 0.2, 0.6]]

np.random.seed(1234)
A = sbm(n=ns, p = B)

# the true community labels
z = [0 for i in range(0,ns[0])] + [1 for i in range(0, ns[1])] + [2 for i in range(0, ns[2])]

In [ ]:
draw_multiplot(A, labels=z, xticklabels=10, yticklabels=10, title="A, Simulated $SBM_{100}( \\vec z, B)$");

Remember, however, that you do not *actually* know the community labels of each node in $A$, so this problem is a little more difficult than it might seem. If you reordered the nodes, the community each node is assigned to would not be as visually obvious as it is here in this example, as you saw in the [introduction](link?).

Our goal is to learn about the block matrix, $B$, which is the parameter that you care about for the SBM. However, you cannot just plug $A$ into the `SBMEstimator` class like you did back when you [fit an SBM using MLE](link?). This is because the `SBMEstimator` uses node community assignments, which you do not have. Instead, what you will do is turn again to the spectral embedding.

You begin by first embedding $A$ to estimate a latent position matrix. Remember that to deduce whether there is any latent structure, you take a look at the latent position matrix using a pairplot, like we learned about in [Chapter 6](#link?):

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed

ase = AdjacencySpectralEmbed()  # adjacency spectral embedding, with optimal number of latent dimensions selected using elbow picking
Xhat = ase.fit_transform(A)

In [ ]:
from graspologic.plot import pairplot

_ = pairplot(Xhat, title="Pairplot of embedding of $A$")

You have some *really* prominent latent structure here, which is evident from the tightly clustered blobs you have in your pairplot. To see just how "tight" these blobs are, let's cheat for a second and look at the true labels of each of the points:

In [ ]:
_ = pairplot(Xhat, labels=z, title="Pairplot of embedding of $A$", legend_name="Community (Unknown)")

So from what you can see, it's pretty clear that these extremely tight "blobs" in your dataset correspond *exactly* to the true community labels of the nodes in your network. So, what you need is a technique which can take a latent position matrix, not know *anything* about these blobs, and and provide us with a way to learn which points correspond to which blob. For this, we turn to unsupervised learning.

### Unsupervised learning via k-means clustering

To learn about these blobs, or more specifically, *clusters* in your dataset, you need a technique which will do the following:

+ **Given**: Estimates of latent positions, $\vec x_i$, for each node $i$ from $1$ to $n$, and a number of clusters to search for, $K$.
+ **Output**: Predicted labels $z_i$ for each node $i$ from $1$ to $n$.

The way you will do this is using an unsupervised clustering technique known as $k$-means clustering. **Unsupervised learning** is the process of learning latent (unknown ahead of time) structure from a dataset. Our problem is *unsupervised* here because you do not give the algorithm any information as to the structure of which nodes are in which communities. 

Through $k$-means, you try to find reasonable guesses at the "centers" of the blobs of latent structure in the dataset. You predict that the label of the point is the center which it is closest to. To do this, you first need a definition of close. Throughout this book, we've already come across one such definition which will do just fine for now: the Euclidean distance. Remember that the Euclidean distance between two points $\vec x_i$ and $\vec x_j$ which each have $d$-total dimensions is the quantity:
\begin{align*}
    ||\vec x_i - \vec x_j||_2 &= \sqrt{\sum_{l = 1}^d (x_{il} - x_{jl})^2}
\end{align*}

To illustrate what's happening graphically, we're going to take a look at a pairsplot of the second and third dimensions, and work through one loop of the $k$-means algorithm. The second and third dimensions look like this:

In [ ]:
from pandas import DataFrame
import seaborn as sns
data = DataFrame({"Dimension 2" : Xhat[:,1], "Dimension 3" : Xhat[:,2]})
palette = {"0" : "blue", "1": "green", "2": "red"}
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data, x="Dimension 2", y="Dimension 3", color="gray", ax=ax)
ax.set_title("Estimates of latent positions");

To perform $k$-means, you need a "starting point" for the centers you will attempt to identify. There are some strategies for doing this strategically (`sklearn`, for instance, uses one called `kmeans++`, which you can read about [on wikipedia](https://en.wikipedia.org/wiki/K-means_clustering#initialization-methods)), but for our purposes we're going to put your centers in basically the worst possible locations: we'll put them smack dab in the middle of nowhere on your graph. This is problematic for $k$-means because of how the "update" step works, but as we'll see in a few minutes, this really isn't going to matter in practice for us:

In [ ]:
centers = np.array([[.5, .5], [-0.05, 0.05], [-0.05, -0.05]])
datcenters = DataFrame({"Dimension 2": centers[:,0], "Dimension 3": centers[:,1], "Cluster": ["0", "1","2"]})
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data, x="Dimension 2", y="Dimension 3", color="gray", ax=ax)
sns.scatterplot(data=datcenters, x="Dimension 2", y="Dimension 3", hue="Cluster",
                palette=palette, ax=ax, s=200)
ax.set_title("Estimates of latent positions with initialized centers");

Next, what you do is you identify which points are "closest" to which cluster center. You do this by computing the distance between each estimate of a latent position and the three centers, and then decide which is the smallest. In the below plot, we recolor each gray point based on which of the centers it is closest to. This is called the points *assignment* to a particular cluster:

In [ ]:
from scipy.spatial import distance_matrix
distances = distance_matrix(Xhat[:,1:3], centers)
assignment = np.argmin(distances, axis=1)

data["Closest Center"] = assignment.astype(str)
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data, x="Dimension 2", y="Dimension 3", hue="Closest Center", palette=palette,
                ax=ax, legend=False)
sns.scatterplot(data=datcenters, x="Dimension 2", y="Dimension 3", hue="Cluster", ax=ax, 
                palette=palette, s=200)
ax.set_title("Temporary cluster assignments for estimates of latent positions");

Which is not too bad! It looks like we've done a pretty good job at getting some of these blobs assigned to similar clusters when points are in the same blob, but you still have some problems. As you can see, some of your blobs have points which are really similar being colored differently, which is unideal. For this reason, we're going to do it all again!

Next, you *update* your centers, by taking the mean value (for each dimension) of the points which are assigned to that cluster. Our centers update like this:

In [ ]:
centers = np.array([np.mean(Xhat[assignment == k,1:3], axis=0) for k in range(0, 3)])

datcenters = DataFrame({"Dimension 2": centers[:,0], "Dimension 3": centers[:,1], "Cluster": ["0", "1","2"]})
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data, x="Dimension 2", y="Dimension 3", hue="Closest Center", palette=palette,
                ax=ax, legend=False)
sns.scatterplot(data=datcenters, x="Dimension 2", y="Dimension 3", hue="Cluster",
                palette=palette, ax=ax, s=200)
ax.set_title("Cluster centers updated based on average of assigned points");

You can see where we're going with this, right? You assume again that the points are totally unlabeled (gray), you recompute which center each point is closest to, and then you re-update your centers. This is called the second *iteration* of the algorithm, because you are doing the exact same process a second time:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 4))
distances = distance_matrix(Xhat[:,1:3], centers)
assignment = np.argmin(distances, axis=1)
centers_new = np.array([np.mean(Xhat[assignment == k,1:3], axis=0) for k in range(0, 3)])

data["Closest Center"] = assignment.astype(str)

color_kwarg = [{"color": "gray"}, {"hue": "Closest Center"}, {"hue": "Closest Center"}]
cdat = [centers, centers, centers_new]
titles = ["1. Centers from previous iteration", "2. Temporary cluster assignments", "3. Update centers"]

for i, ax in enumerate(axs.flat):
    sns.scatterplot(data=data, x="Dimension 2", y="Dimension 3", ax=ax, **color_kwarg[i],
                    palette=palette, legend=False)
    datcenters = DataFrame({"Dimension 2": cdat[i][:,0], "Dimension 3": cdat[i][:,1], "Cluster": ["0", "1","2"]})
    sns.scatterplot(data=datcenters, x="Dimension 2", y="Dimension 3", hue="Cluster",
                    palette=palette, ax=ax, s=200)
    ax.set_title(titles[i]);

And you just keep repeating this 3 step procedure over and over again, until your centers stop changing very much. When you stop again is an area of research interest like the initialization procedure for the centers, known as the *stopping criterion* for the algorithm. As you can see, after doing this process just *twice*, you already have homogeneous blobs, in that points from each of the three clusters are all assigned to the same cluster. You can automate this entire process and use the nearest clusters to produce the "predicted labels" using `sklearn`:

In [ ]:
from sklearn.cluster import KMeans

labels_kmeans = KMeans(n_clusters = 3, random_state=1234).fit_predict(Xhat)

You can take a look at the pairs plot now, with the *predicted labels* and check to ensure that you found the blobs you saw visually as uniform clusters:

In [ ]:
_ = pairplot(Xhat, labels=labels_kmeans, title="Pairplot of embedding of $A$", legend_name="Predicted Cluster")

You use these predicted cluster labels as the *predicted communities* for your Stochastic Block Model.

### Evaluating $k$-means clustering

Since you have simulated data, you have the benefit of being able to evaluate the quality of your predicted community assignments to the true community assignments. However, there is an important caveat: your predictions might not *necessarily* align with your true labels. What we mean by this is that, your true communities might impart something meaningful about your dataset (here, they are just 0s, 1s, and 2s, but in a real dataset, they could be more meaningful things, like "School 1", "School 2", and "School 3"). Since we did not assume you knew *anything* about the communities when you ran your clustering, the predicted clusters will not have a correspondance with the original community names. In your example, this means that even though community 0 in the true labels and community 2 in the predicted labels encompass the same points, since the algorithm didn't know to call those points community 0, it just arbitrarily called them community 2. How can you proceed?

#### The confusion matrix lets us visualize the homogeneity of predictions relative the true labels

To overcome this limitation for evaluation, we look at something called a confusion matrix. A **confusion matrix** is a matrix you use when you have two sets of labels for a group of data points, one of which you know to be the *true* labels, and another set of labels for which you do not know (yet) whether there is a correspondance with the true set of labels. If the set of true labels is one of $L$ possible values and the other set of labels take one of $K$ possible values, the confusion matrix will be:

| True label | Predicted label $1$ | ... | Predicted label $K$ |
| --- | --- | --- | --- |
| True label $1$ | #points(Predicted label $1$, true label $1$) | ... | #points(Predicted label $K$, true label $1$) |
| True label $2$ | #points(Predicted label $1$, true label $2$) | ... | #points(Predicted label $K$, true label $2$) |
| ... | ... | ... | ... |
| True label $L$ | #points(Predicted label $1$, true label $L$) | ... | #points(Predicted label $K$, true label $L$) |

You can compute and plot this matrix using a heatmap:

In [ ]:
from sklearn.metrics import confusion_matrix
# compute the confusion matrix between the true labels z
# and the predicted labels labels_kmeans
cf_matrix = confusion_matrix(z, labels_kmeans)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.heatmap(cf_matrix, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Confusion matrix")
ax.set_ylabel("True Label")
ax.set_xlabel("Predicted Label");

What you want to see, in general, in the confusion matrix for a "good" clustering is that the proportion of nodes which have a particular true label are "largely" assigned to the same predicted label. In this case, it's really obvious, because the predictions are virtually entirely (if not entirely) homogeneous, in that a single predicted label corresponds to a single true label. However, for posterity, a useful exercise is to get used to normalizing this confusion matrix, because things won't always be quite so obvious. You can do this by the counts by the total number of points assigned to a particular true label, which gives us the normalized confusion matrix:

In [ ]:
cfm_norm = cf_matrix/cf_matrix.sum(axis=1)[:,None]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.heatmap(cfm_norm, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Normalized confusion matrix")
ax.set_ylabel("True Label")
ax.set_xlabel("Predicted Label");

Now, it becomes clear that all of the points with a true label of $0$ are assigned to a predicted label of $1$, so on and so forth for all of the individual rows. So, how do you actually determine whether a confusion matrix indicates you did a good job with your predictions, and that your predictions and true labels "align"? 

#### You can evaluate the homogeneity of the confusion matrix using the adjusted rand index (ARI)

For this purpose, we turn to the Rand Index, or RI. With the rand index, instead of looking at the confusion matrix itself *directly*, you look at all pairs of your data points. For a given pair of points in your dataset, these points can have true labels that are either the same, or different. Further, these points can have predicted labels that are either the same, or different. We introduce the following two concepts:
1. *Success*: Two points with the same true labels have the same predicted labels, or two points with different true labels have different predicted labels. This means that the two points *align homogeneously* across the true and predicted labels.
2. *Failure*: Two points with different true labels have the same predicted labels, or two points with the same true labels have different predicted labels. This means that the two points have a *heterogeneous alignment* across the true and predicted labels.
To compute the rand index, you simply take the ratio of successes divided by the total number of comparisons:
\begin{align*}
    RI &= \frac{\text{Successes}}{\text{Successes} + \text{Failures}}
\end{align*}

The Rand Index will have a value between $0$ or $1$, where a high value corresponds to a majority of the comparisons being successes (and hence, the true labels and predicted labels are homogeneous). 

There's a slight problem here when you want to use this number for evaluating a clustering. What if your true labels are disproportionate, in that one true label has a ton of data points, and the others only a small fraction of data points? By simply *guessing* that all of the points are from the biggest class, you could still end up with a pretty high RI. 

Consider, for instance, a simple example example in the very extreme case. Let's say that you have $10$ points, $9$ of which are in class $1$, and $1$ of which is in class $2$. What happens to the RI if you were to learn nothing about the structure of these points, and just guess that all of the points are in class $1$ without even looking at the data?

You can pick a single point from $10$ possible points, and then you compare this point to any of the other $9$ points. This means you will have $10 \cdot 9 = 90$ comparisons in total. Let's break down how these comparisons will go. You first choose the first point, which can be one of the $9$ items in class $1$, or the $1$ item in class $2$:
1. The first point is in class $1$:
    + The second point is also in class $1$: Of the remaining $9$ points you could choose for the second point, $8$ of them will have the same true label and predicted label as point $1$ (since you always guessed items were from class $1$). This means you will have $9$ (number of possible points which are from class $1$) times $8$ (number of remaining points which also have a true label of class $1$) $=72$ successes in which the point has a true label of $1$ and a predicted label of $1$.
2. The first point is in class $2$:
    + You have no successful comparisons when the first point is in class $2$, since the other points all have the same predicted label but a different true label.

This means that your RI is $\frac{72}{90} = 0.8$, which *feels* really high doesn't it, since it's closer to the highest possible value of $1$ than the lowest possible value of $0$?

To overcome this "weakness" in the RI, you instead "adjust" for the possible situations like this where the fractions of points with a particular true label are unevenly dispersed throughout the dataset. This "adjustment" to the RI is aptly named: it is called the **Adjusted Rand Index** (ARI). You can compute the ARI using `sklearn` easily:

In [ ]:
from sklearn.metrics import adjusted_rand_score

ari_kmeans = adjusted_rand_score(z, labels_kmeans)

In [ ]:
print("ARI(predicted communities, true communities) = {}".format(ari_kmeans))

#### By remapping the labels, you can evaluate the error rate

Looking back at the confusion matrix:

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.heatmap(cf_matrix, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Confusion matrix")
ax.set_ylabel("True Label")
ax.set_xlabel("Predicted Label");

It seems pretty clear that the label correspondance between the true labels and predicted labels is as follows:
1. Nodes which have a true label of $0$ correspond to a predicted label of $0$,
2. Nodes which have a true label of $1$ correspond to a predicted label of $2$,
3. Nodes which have a true label of $2$ correspond to a predicted label of $1$.
This is because all of the nodes which have a true label of $0$ are assigned a predicted label of $0$, all of the nodes with a true label of $1$ are assigned a predicted label of $2$, and all of the nodes with a true label of $2$ have a predicted label of $1$. But when the results aren't perfect, this is a little bit harder to do, and involves a bit of background in optimization theory, which is outside of the scope of this book. Fortunately, `graspologic` makes this easy for us, with the `remap_labels` function:

In [ ]:
from graspologic.utils import remap_labels

labels_kmeans_remap = remap_labels(z, labels_kmeans)

You can use these remapped labels to understand when `KMeans` is, or is not, producing reasonable labels for your investigation. You begin by first looking at a pairs plot, which now will color the points by their assigned community:

In [ ]:
_ = pairplot(Xhat,
         labels=labels_kmeans_remap,
         title=f'KMeans on embedding, ARI: {str(ari_kmeans)[:5]}',
         legend_name='Predicted label (remapped)',
         height=3.5,
         palette='muted',);

The final utility of the pairs plot is that you can investigate which points, if any, the clustering technique is getting wrong. You can do this by looking at the classification error of the nodes to each community:

In [ ]:
error = z - labels_kmeans_remap  # compute which assigned labels from labels_kmeans_remap differ from the true labels z
error = error != 0  # if the difference between the community labels is non-zero, an error has occurred
er_rt = np.mean(error)  # error rate is the frequency of making an error

palette = {'Correct Pred.':(0,0.7,0.2),
           'Incorrect Pred.':(0.8,0.1,0.1)}

error_label = np.array(len(z)*['Correct Pred.'])  # initialize numpy array for each node
error_label[error] = 'Wrong Pred.'  # add label 'Wrong' for each error that is made

In [ ]:
pairplot(Xhat,
         labels=error_label,
         title='Error from KMeans, Error rate: {:.3f}'.format(er_rt),
         legend_name='Error label',
         height=3.5,
         palette=palette,);

Great! Our classification has not made any errors.

To learn about the block matrix $B$, you can now use the `SBMEstimator` class, with your predicted labels:

In [ ]:
from graspologic.models import SBMEstimator

model = SBMEstimator(directed=False, loops=False)
model.fit(A, y=labels_kmeans_remap)
Bhat = model.block_p_

In [ ]:
from graspologic.plot import heatmap

fig, axs = plt.subplots(1, 3, figsize=(18, 6))

mtxs = [Bhat, np.array(B), np.abs(Bhat - np.array(B))]
titles = ["True $B_{SBM}$", " Prediction $\hat B_{SBM}$", "$|\hat B_{SBM} - B_{SBM}|$"]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i],
            vmin=0,
            vmax=1,
            font_scale=1.5,
            title=titles[i],
            ax=ax, cmap=cmaps["sequential"])

Which appears to be relatively close to the true block matrix.

```{admonition} Recap of inference for Stochastic Block Model with known number of communities

1. You learned that the adjacency spectral embedding is a key algorithm for making sense of networks which are realizations of SBM random networks. The estimates of latent positions produced by ASE are critical for learning community assignments.
2. You learned that unsuperised learning (such as K-means) allows us to ues the estimated latent positions to learn community assignments for each node in your network. 
3. You can use `remap_labels` to align predicted labels with true labels, if true labels are known. This is useful for benchmarking techniques on networks with known community labels.
4. You evaluate the quality of unsupervised learning by plotting the predicted node labels and (if you know the true labels) the errorfully classified nodes. The ARI and the error rate summarize how effective your unsupervised learning techniques performed.
```

## Number of communities $K$ is not known

In real data, you almost never have the beautiful canonical modular structure obvious to yourom a Stochastic Block Model. This means that it is *extremely infrequently* going to be the case that you know exactly how you should set the number of communities, $K$, ahead of time.

Let's first remember back to the single network models section, when you took a Stochastic block model with obvious community structure, and let's see what happens when you just move the nodes of the adjacency matrix around. You begin with a similar adjacency matrix to $A$ given above, for the $3$-community SBM example, but with the within and between-community edge probabilities a bit closer together so that you can see what happens when you experience errors. The communities are still quite apparent from the adjaceny matrix:

In [ ]:
B = [[0.4, 0.2, 0.2],
     [0.2, 0.5, 0.2],
     [0.2, 0.2, 0.4]]
ns = [40, 40, 40]
np.random.seed(12)
A = sbm(n=ns, p = B)

# the true community labels
z = [0 for i in range(0,ns[0])] + [1 for i in range(0, ns[1])] + [2 for i in range(0, ns[2])]
draw_multiplot(A, labels=z, xticklabels=10, yticklabels=10, title="Simulated SBM($\pi$, B)");

Next, you permute the nodes around to reorder the realized adjacency matrix:

In [ ]:
# generate a reordering of the n nodes
vtx_perm = np.random.choice(A.shape[0], size=A.shape[0], replace=False)

A_permuted = A[tuple([vtx_perm])] [:,vtx_perm]
z_perm = np.array(z)[vtx_perm]

In [ ]:
from graphbook_code import heatmap as hm_code 
from graphbook_code import draw_layout_plot as lp_code

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# heatmap
hm = hm_code(
    A_permuted,
    ax=axs[0],
    cbar=False,
    color="sequential",
    xticklabels=10,
    yticklabels=10
)

# layout plot
lp_code(A_permuted, ax=axs[1], pos=None, labels=z_perm, node_color="qualitative")
plt.suptitle("Simulated $SBM(\\vec z, B)$, reordered vertices", fontsize=20, y=1.1)

fig;

You only get to see the adjacency matrix in the *left* panel; the panel in the *right* is constructed by using the true labels (which you do *not* have!). This means that you proceed for statistical inference about the random network underlying your realized network using *only* the heatmap you have at left. It is not immediately obvious that this is the realization of a random network which is an SBM with $3$ communities.

Our procedure is *very* similar to what you did previously [when the number of communities was known](#link?). You again embed using the "elbow picking" technique:

In [ ]:
ase_perm = AdjacencySpectralEmbed()  # adjacency spectral embedding, with optimal number of latent dimensions selected using elbow picking
Xhat_permuted = ase_perm.fit_transform(A_permuted)

You examine the pairs plot, *just* like in the section on [pairs plots](#link?):

In [ ]:
_ = pairplot(Xhat_permuted, title="SBM adjacency spectral embedding")

You can still see that there is some level of latent community structure apparent in the pairs plot, as you can see some "blob" separation.

Next, you have the biggest difference with the approach you took previously. Since you do *not* know the optimal number of clusters $K$ *nor* the true community assignments, you must choose an unsupervised clustering technique which allows us to *compare* clusterings with different choices of cluster counts. Fortunately, this is pretty easy for us to do, too, using a simple statistic known as the Silhouette score.

### The Silhouette score allows us to compare unsupervised clustering quality

The Silhouette score will provide you with a statistic that you can compare across clusterings with different numbers of clusters. At an extremely high level, the Silhouette score asks the simple question, "How similar are the nodes in the same cluster from other clusters?" If for a given number of clusters, and those clusters are *spatially unique* (there aren't too many clusters for what is really just a single blob of points, and likewise, there aren't single clusters doing a job that might be better served by two clusters), then the Silhouette score will, ideally, be higher. We mention this statistic in particular because it can be reasonably used with *any* unsupervised learning technique, and won't pose restrictions on the unsupervised technique chosen, for instance. Other popular techniques, such as the Bayesian Information Criterion (BIC), can *only* be computed when assumptions are made about the unsupervised learning technique used, but you can read about them on your own time for more information if you want.

To compute the Silhouette score, you need some basic ingredients first. You have data points $x_i$ for $i$ from $1$ to $n$, the estimates of latent positions of the nodes in your network, and you have predicted community labels $z_i$, which assign each node to one of $K$ possible clusters. We will call the set of nodes assigned to the cluster with label $k$ the set $C_k$, which is just a collection of indexes of nodes in the network. For instance, if the network had $10$ nodes and the total number of clusters you were considering was two, where the first five nodes were assigned to cluster $1$ and the second five nodes were assigned to cluster $2$, then $C_1 = \{1,2,3,4,5\}$, and $C_2 = \{6,7,8,9,10\}$. We will call the total number of nodes which are in community $k$ the quantity $n_k$. You have the following two ingredients:

1. The dissimilarity of a node $i$ from the other nodes in its community: You compute the average distance between the estimate of the latent position for the node $i$, and all of the other nodes which are in the same community as node $i$. If you assume that the node $i$ is in community $k$, this is just the quantity:
\begin{align*}
    a_i &= \frac{1}{n_k - 1} \sum_{j \in C_k, i \neq j}||\vec x_i - \vec x_j||
\end{align*}
where $||x - y||$ is the Euclidean distance between two vectors $\vec x_i$ and $\vec x_j$. The sum just indicates that you are computing distances over all of the other nodes in community $k$ which are *not* node $i$ (node $i$ is in community $k$, so there are $n_k - 1$ of these other points), and then you take the average by dividing by the number of points in community $k$ which are *not* node $i$.
2. The dissimilarity of a node $i$ from the other nodes in a different community: We compute the average distance between the estimate of the latent position for the node $i$, and this time all of the other nodes which are in some other community $l$ from node $i$. If you assume that the node $i$ is in community $k$, this is just the quantity:
\begin{align*}
    b_{il} &= \frac{1}{n_l} \sum_{j \in C_l}||\vec x_i - \vec x_j||
\end{align*}
Node $i$ is in community $k$, so this time, you don't need to worry about excluding the node $i$ from your comparisons (and, you can compare to every other node in $C_l$).
3. The *other* community that node $i$ is most similar to: This is just the community for which has the *smallest* dissimilarity from node $i$, and hence, is the *best alternative cluster* that you could have assigned $i$ to:
\begin{align*}
    d_i &= \min_{l\text{ is a community}}b_{il}
\end{align*}
And the Silhouette of node $i$ in community $k$ is just:
\begin{align*}
    s_i &= \begin{cases}
        \frac{d_i - a_i}{\max(a_i, d_i)} & n_k > 1 \\
        0 & n_k = 1
    \end{cases}
\end{align*}
In words, if the Silhouette $s_i$ is near $1$, then the node $i$ is much more similar to points which are in its own cluster than the best alternative, since since a value near $1$ can only happen if $d_i$ is much bigger than $a_i$. Further, if the Silhouette is near $-1$, then the node $i$ is more similar to points which are in a neighboring cluster which is not the one it was assigned to. This is because negative values can only happen if node $i$s dissimilarity from other points in its cluster, $a_i$, is higher than its dissimilarity from other points in another cluster, $d_i$. 

You get the silhouette score for however many clusters you are trying $K$ by just taking the average of the node-wise silhouettes:
\begin{align*}
    S_K &= \frac{1}{n} \sum_{i = 1}^n s_i
\end{align*}

### Using the Silhouette score to deduce an appropriate number of communities

Now that you have the Silhouette score, deducing an appropriate number of communities is pretty easy. You choose a range of clusters that you think might be appropriate for your network. Let's say, for instance, you think there might be as many as $10$ clusters in your dataset. You perform a clustering using unsupervised learning for all possible numbers of clusters, from $2$ all the way up to the maximum number of clusters you think could be reasonable. Then, you compute the Silhouette score for this number of clusters. Finally, you choose the number of clusters which has the highest Silhouette score. Let's see how to do this, using `graspologic`'s `KMeansCluster` function:

In [ ]:
from graspologic.cluster import KMeansCluster

km_clust = KMeansCluster(max_clusters = 10, random_state=1234)
km_clust = km_clust.fit(Xhat_permuted);

Next, you visualize the Silhouette Score as a function of the number of clusters:

In [ ]:
import seaborn as sns
from pandas import DataFrame as df

nclusters = range(2, 11)  # graspologic nclusters goes from 2 to max_clusters
silhouette = km_clust.silhouette_ # obtain the respective silhouette

bic_df = df({"Number of Clusters": nclusters, "Silhouette Score": silhouette})  # place into pandas dataframe

fig, ax = plt.subplots(1,1,figsize=(12, 6))

sns.lineplot(data=bic_df,ax=ax, x="Number of Clusters", y="Silhouette Score");
ax.set_title("Silhouette Analysis of KMeans Clusterings")
fig;

As you can see, Silhouette analysis has indicated the best number of clusters as $3$ (which, is indeed, *correct* since we are performing a simulation where we know the right answer). We can get the labels produced by $k$-means using automatic number of clusters selection as follows:

In [ ]:
labels_autokmeans = km_clust.fit_predict(Xhat_permuted)

As an exercise, you should go through the preceding section, and recompute all the diagnostics we did last time for determining whether our predictions were reasonable. You will do this by comparing the true permuted labels `y_perm` to the predicted labels on the permuted nodes, `labels_autokmeans`.

## Community detection with other types of embeddings

If you remember from Chapter 6, we also discussed several other embedding techniques, such as [LSE](#link?), [node2vec](#link?), and [GNNs](#link?). These techniques can be used, often interchangeably, with `ASE` as we discussed above to produce similar predicted community assignments for your network, or different depending on the parameters chosen for the embedding technique and the topology of the network. For instance, in the section on the "two-truths" phenomena from [Chapter 6](#link?), you saw an example where `ASE` and `LSE` produced embeddings which yielded different blobs of nodes, where `ASE` separated nodes based on whether the nodes had affinity structure from one another, and `LSE` separated nodes based on whether they were core or periphery nodes.

### Can you detect "two-truths" with one embedding?

As a fun exercise, let's consider an example of an algorithm which can give you latent structure for *both* truths in the two-truths example, `node2vec`. Let's get you an adjacency matrix with both affinity and core-periphery structure to work on:

In [ ]:
import graspologic as gp

n1 = 50; n2 = 50
Baff = [[0.5, 0.2], [0.2, 0.5]]

zvecaff = np.array(["C1" for i in range(0, n1)] + ["C2" for i in range(0, n2)])

# the probability matrix
zvecaff_ohe = np.vstack([[1, 0] for i in range(0, n1)] + [[0, 1] for i in range(0, n2)])
Paff = zvecaff_ohe @ Baff @ zvecaff_ohe.T

np1 = 15; nc = 70; np2 = 15
Bcp = [[0.1, 0.1, 0.1],
      [0.1, 0.7, 0.1],
      [0.1, 0.1, 0.1]]


zveccp = np.array(["Per." for i in range(0, np1)] +
                  ["Core" for i in range(0, nc)] +
                  ["Per." for i in range(0, np2)])

# the probability matrix
zveccp_ohe = np.vstack([[1, 0, 0] for i in range(0, np1)] + 
                       [[0, 1, 0] for i in range(0, nc)] +
                       [[0, 0, 1] for i in range(0, np2)])
Pcp = zveccp_ohe @ Bcp @ zveccp_ohe.T

P_aff_and_cp = (Pcp + Paff)/2
np.random.seed(1234)
A_aff_and_cp =  gp.simulations.sample_edges(P_aff_and_cp, directed=False, loops=False)

The probability matrix and the adjacency matrix look like this:

In [ ]:
from graphbook_code import cmaps

fig, axs = plt.subplots(1,2, figsize=(16,9))

gp.plot.heatmap(P_aff_and_cp, ax=axs[0], inner_hier_labels=zveccp, outer_hier_labels=zvecaff,
                title="", vmin=0, vmax=1, cmap=cmaps["sequential"]);
gp.plot.heatmap(A_aff_and_cp, ax=axs[1], inner_hier_labels=zveccp, outer_hier_labels=zvecaff,
                title="", vmin=0, vmax=1, cmap=cmaps["sequential"]);

You embed it into $4$ dimensions using `node2vec` like this, with true affinity community labels first:

In [ ]:
from graspologic.embed import node2vec_embed as node2vec
import networkx as nx

p = 0.1; q = 10; w = 20; T = 200
d = 4  # number of embedding dimensions to use
Xhat, _ = node2vec(nx.from_numpy_matrix(A_aff_and_cp), return_hyperparameter=float(p),
                     inout_hyperparameter=float(q), dimensions=d, num_walks=w, walk_length=T,
                     iterations=200, random_seed=1234)

In [ ]:
from graspologic.plot import pairplot

_ = pairplot(Xhat, labels=zvecaff, title="Node2Vec Embedding of 2-Truths example", legend_name="Affinity")

Seems like we've got two relatively separate blobs, so let's see if we can identify them using `KMeans` with $2$ clusters like we did previously. Nothing too challenging here, just review of what we've already done:

In [ ]:
kmeans_n2v_aff = KMeans(n_clusters=2)
preds_aff = kmeans_n2v_aff.fit_predict(Xhat)

We visualize the predictions as a confusion matrix (the `confusion_matrix()` function can only handle the cases where both the truths and the predictions are integers or strings, not mixed types, so we have to convert the predicted labels first), and calculate the ARI and error rate:

In [ ]:
# arbitrary call C1 0 and C2 1
map_truth_to_binary = {"C1": 0, "C2": 1}

# and convert zvecaff to the binary map we specified above
zvecaff_binary = [map_truth_to_binary[i] for i in zvecaff]
# confusion matrix and normalized confusion matrix
cf_mtx_n2v_aff = confusion_matrix(zvecaff_binary, preds_aff)
cf_norm_n2v_aff = cf_mtx_n2v_aff/cf_mtx_n2v_aff.sum(axis=1)[:,None]
ari = adjusted_rand_score(zvecaff, preds_aff)

# remap the predicted labels to the binarized affinity labels
preds_aff_remapped = remap_labels(zvecaff_binary, preds_aff)
er_rt = np.mean(preds_aff_remapped != zvecaff_binary)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.heatmap(cf_norm_n2v_aff, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Normalized confusion matrix, ARI={:.3f}, Error = {:.3f}".format(ari, er_rt))
ax.set_ylabel("True Label")
ax.set_xlabel("Predicted Label");

So we have a pretty decent ARI of $0.514$, and our error rate is only $0.14$. 

#### Sometimes, you have to modify your data to learn from it

Now, what to do about the core-periphery nodes? If we peek at the pairs plot with the core-periphery nodes highlighted, we see:

In [ ]:
_ = pairplot(Xhat, labels=zveccp, title="Node2Vec Embedding", legend_name="Core-Periphery")

For some of these dimensions, for instance, dimension $1$ against dimension $2$, we can see a sort of "disc" forming which separates the points pretty well into the core versus the peripheral nodes. To make these points more separable based on this fact is pretty easy. Let's start by taking a look at the points we have, and we'll show where the center of these points is with a black star:

In [ ]:
# compute the center of each dimension
center = np.mean(Xhat, axis=0)

In [ ]:
data_n2v = DataFrame({"Dimension 2" : Xhat[:,1], "Dimension 3" : Xhat[:,2], "Core-Periphery": zveccp})
palette = {"Core": "blue", "Per.": "red"}
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data_n2v, x="Dimension 2", y="Dimension 3", 
                hue="Core-Periphery", palette=palette, alpha = 0.5,ax=ax)
ax.set_title("Node2Vec Embedding of 2-Truths Example");
ax.plot(center[1], center[2], color="black", markersize=20, alpha=1, marker="*");

The nodes close to the star tend to be core nodes, and the nodes farther away from the star tend to be peripheral nodes. So, what happens if we just measure how far each point is away from this center, and then use that as our coordinate system instead? Let's give that a try:

In [ ]:
Xhat_xfm = np.abs(Xhat - center)

In [ ]:
data_n2v_xfm = DataFrame({"Distance from center in dimension 2" : Xhat_xfm[:,1],
                          "Distance from center in dimension 3" : Xhat_xfm[:,2],
                          "Core-Periphery": zveccp})
palette = {"Core": "blue", "Per.": "red"}
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(data=data_n2v_xfm, x="Distance from center in dimension 2", y="Distance from center in dimension 3", 
                hue="Core-Periphery", palette=palette, alpha = 0.5,ax=ax)
ax.set_title("Node2Vec Embedding of 2-Truths Example");

Now, we're starting to get somewhere! Let's look at the pairs plot now:

In [ ]:
_ = pairplot(Xhat_xfm, labels=zveccp, title="Embedding after transforming for distance to center", legend_name="Core-Periphery")

Things are starting to look a lot more obvious! Now, we have all of the core nodes pretty well-separated from the peripheral nodes. We'll classify these points using Gaussian Mixture Model, which is similar to `KMeans`, but allows us to capture the fact that the points from the Peripheral community are more spread out than the points from the core community:

In [ ]:
from sklearn.mixture import GaussianMixture

preds_cp = GaussianMixture(n_components=2, random_state=1234).fit_predict(Xhat_xfm)

We repeat our analysis that we did previously to obtain the confusion matrix, the ARI, and the error rate:

In [ ]:
# arbitrary call Core 0 and Per. 1
map_truth_to_binary = {"Core": 0, "Per.": 1}
map_binary_to_truth = {0: "Core", 1: "Per."}

# and convert zveccp to the binary map we specified above
zveccp_binary = [map_truth_to_binary[i] for i in zveccp]
# confusion matrix and normalized confusion matrix
cf_mtx_n2v_cp = confusion_matrix(zveccp_binary, preds_cp)
cf_norm_n2v_cp = cf_mtx_n2v_cp/cf_mtx_n2v_cp.sum(axis=1)[:,None]
ari = adjusted_rand_score(zveccp, preds_cp)

# remap the predicted labels to the binarized core-periphery labels
preds_cp_remapped = remap_labels(zveccp_binary, preds_cp)
er_rt = np.mean(preds_cp_remapped != zveccp_binary)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.heatmap(cf_norm_n2v_aff, cmap=cmaps["sequential"], ax=ax)
ax.set_title("Normalized confusion matrix, ARI={:.3f}, Error = {:.3f}".format(ari, er_rt))
ax.set_ylabel("True Label")
ax.set_xlabel("Predicted Label");

Wow! We were only wrong $7\%$ percent of the time! This is incredible! To emphasize just how crazy this is, let's look back at the points, before we did any of this tranformation business, and look at the predictions we were able to produce. We'll use `remap_labels` on our predictions so we can make more sense of them:

In [ ]:
preds_cp_remapped = [map_binary_to_truth[j] for j in remap_labels(zveccp_binary, preds_cp)]

In [ ]:
_ = pairplot(Xhat, labels=preds_cp_remapped, title="Node2Vec Embedding", legend_name="Prediction")

Which looks like our classifier was able to successfully capture the idea that the core nodes tend to be in the middle, and the peripheral nodes around the edge!

This example is a great exercise in why you should always look carefully and think about your data before you go ahead and analyze it. If you hadn't looked at this pair plot, the fact that the core nodes tended to be closer to the center would have been completely unobvious to you. Further, if you hadn't *thought* about this fact, you wouldn't have known how you could *exploit* this trend in your data by just taking the distance to the center in each dimension. Notice that other than this simple transformation, you used basically the same strategies from before; you didn't need to do anything fancy with your clustering algorithm, and you were able to make predictions for an otherwise quite complicated embedding by just changing the dimensions of our embedding around slightly. 